# Data Analysis of Olivocerebellar Model

## Table of Contents
### 1. Import All
    1. Import packages
    2. Import data of simulations
### 2. Split Data - Before/After Adaptation
### 3. 

## 1. Import All

In [ ]:
# 1.1) Import packages
import numpy as np
from brian2 import*
import NeuroTools as nt
import matplotlib as plt
from scipy.signal import find_peaks
import scipy.io as sio
class Struct:
    pass

# 1.2) Import data of simulations
nameInput = 'NoiseDS_test2'
name_simparam = nameInput + '_SimParams'
name_VoltCell = nameInput + '_VoltageCell'
name_SpikeT = nameInput + '_SpikeTimes'
name_PV = nameInput + '_PlasticityVariables'
name_Input = nameInput + 'AfterSim'

SimParam = sio.loadmat(name_simparam)
VoltCell = sio.loadmat(name_VoltCell)
SpikeTimes = sio.loadmat(name_SpikeT)
PlasticityVar = sio.loadmat(name_PV)
Input = sio.loadmat(name_Input)

## 2. Split Data - Before/After Adaptation

In [ ]:
# 2) Split Data - before/after adaptation    

# before = start_t:t_beforeAdaptation
# after = t_beforeAdaptaion:end
# indicated by BA (BeforeAdaptation) and AA (AfterAdaptation) accordingly


start_t = 1000 # remove first second of data due to transient
t_beforeAdaptation = int(len(Input['Input']['I'][0][0][0])/2) # halfway the simulation
nrInputs=Input['Input']['I'][0][0].shape[0] # number of inputs
nrPC = VoltCell['VoltCell']['PC'][0][0].shape[0]
nrDCN = VoltCell['VoltCell']['DCN'][0][0].shape[0]
nrIO = VoltCell['VoltCell']['IOsoma'][0][0].shape[0]
lengthSim =VoltCell['VoltCell']['PC'][0][0].shape[1]
# Input to the system
Input_s = Struct()
Input_s_I = [Input['Input']['I'][0][0][k][start_t:] for k in range(0,nrInputs)]
Input_s.I_BA= [Input['Input']['I'][0][0][k][start_t:t_beforeAdaptation] for k in range(0,nrInputs)]
Input_s.nweight_BA=[Input['Input']['nweight'][0][0][k][start_t:t_beforeAdaptation] for k in range(0,nrInputs)]
Input_s.I_AA =[Input['Input']['I'][0][0][k][t_beforeAdaptation:] for k in range(0,nrInputs)]
Input_s.nweight_AA =[Input['Input']['nweight'][0][0][k][t_beforeAdaptation:] for k in range(0,nrInputs)]

# Membrane potentials of all the different cells
VoltCell_s = Struct()
VoltCell_s.PC = [VoltCell['VoltCell']['PC'][0][0][k][start_t:] for k in range(0,nrPC)]
VoltCell_s.PC_BA = [VoltCell['VoltCell']['PC'][0][0][k][start_t:t_beforeAdaptation] for k in range(0,nrPC)]
VoltCell_s.PC_AA = [VoltCell['VoltCell']['PC'][0][0][k][t_beforeAdaptation:] for k in range(0,nrPC)]
VoltCell_s.DCN = [VoltCell['VoltCell']['DCN'][0][0][k][start_t:] for k in range(0,nrDCN)]
VoltCell_s.DCN_BA =[VoltCell['VoltCell']['DCN'][0][0][k][start_t:t_beforeAdaptation] for k in range(0,nrDCN)]
VoltCell_s.DCN_AA =[VoltCell['VoltCell']['DCN'][0][0][k][t_beforeAdaptation:] for k in range(0,nrDCN)]
VoltCell_s.IOsoma =[VoltCell['VoltCell']['IOsoma'][0][0][k][start_t:] for k in range(0,nrIO)]
VoltCell_s.IOsoma_BA =[VoltCell['VoltCell']['IOsoma'][0][0][k][start_t:t_beforeAdaptation] for k in range(0,nrIO)]
VoltCell_s.IOsoma_AA =[VoltCell['VoltCell']['IOsoma'][0][0][k][t_beforeAdaptation:] for k in range(0,nrIO)]
VoltCell_s.IOdend =[VoltCell['VoltCell']['IOdend'][0][0][k][start_t:] for k in range(0,nrIO)]
VoltCell_s.IOdend =[VoltCell['VoltCell']['IOdend'][0][0][k][start_t:t_beforeAdaptation] for k in range(0,nrIO)]
VoltCell_s.IOdend =[VoltCell['VoltCell']['IOdend'][0][0][k][t_beforeAdaptation:] for k in range(0,nrIO)]
SpikePC=[]

# Spike Times of all cells
SpikeTimes_s = Struct()
for p in range(0,nrPC):
    nr = str(p)
    PCnr = "PC" + nr
    BA_PCnr = 'PC_BA' + nr
    AA_PCnr = 'PC_AA'+ nr
    nrSpikesPC = SpikeTimes['SpikeTimes']['PC'][0][0][0][p][0].shape[0]
    setattr(SpikeTimes_s, PCnr, [SpikeTimes['SpikeTimes']['PC'][0][0][0][p][0][k] for k in range(0,nrSpikesPC) if SpikeTimes['SpikeTimes']['PC'][0][0][0][p][0][k]>=(start_t/1e3)])
    setattr(SpikeTimes_s, BA_PCnr, [SpikeTimes['SpikeTimes']['PC'][0][0][0][p][0][k] for k in range(0,nrSpikesPC) 
                                    if (SpikeTimes['SpikeTimes']['PC'][0][0][0][p][0][k]>=(start_t/1e3) 
                                        and SpikeTimes['SpikeTimes']['PC'][0][0][0][p][0][k]<=(lengthSim/(2*1e3)))])
    setattr(SpikeTimes_s, AA_PCnr, [SpikeTimes['SpikeTimes']['PC'][0][0][0][p][0][k] for k in range(0,nrSpikesPC) 
                                    if SpikeTimes['SpikeTimes']['PC'][0][0][0][p][0][k]>=(t_beforeAdaptation/1e3)])
for p in range(0,nrDCN):
    nr = str(p)
    DCNnr = "DCN" + nr
    BA_DCNnr = 'DCN_BA' + nr
    AA_DCNnr = 'DCN_AA'+ nr
    nrSpikesDCN = SpikeTimes['SpikeTimes']['DCN'][0][0][0][p][0].shape[0]
    setattr(SpikeTimes_s, DCNnr, [SpikeTimes['SpikeTimes']['DCN'][0][0][0][p][0][k] for k in range(0,nrSpikesDCN) 
                                  if SpikeTimes['SpikeTimes']['DCN'][0][0][0][p][0][k]>=(start_t/1e3)])
    setattr(SpikeTimes_s, BA_DCNnr, [SpikeTimes['SpikeTimes']['DCN'][0][0][0][p][0][k] for k in range(0,nrSpikesDCN) 
                                    if (SpikeTimes['SpikeTimes']['DCN'][0][0][0][p][0][k]>=(start_t/1e3) 
                                        and SpikeTimes['SpikeTimes']['DCN'][0][0][0][p][0][k]<=(lengthSim/(2*1e3)))])
    setattr(SpikeTimes_s, AA_DCNnr, [SpikeTimes['SpikeTimes']['DCN'][0][0][0][p][0][k] for k in range(0,nrSpikesDCN) 
                                    if SpikeTimes['SpikeTimes']['DCN'][0][0][0][p][0][k]>=(t_beforeAdaptation/1e3)])

for p in range(0,nrIO):
    nr = str(p)
    IOnr = "IO" + nr
    BA_IOnr = 'IO_BA' + nr
    AA_IOnr = 'IO_AA'+ nr
    findspikes, _ = find_peaks(VoltCell['VoltCell']['IOsoma'][0][0][p], height=0.0, distance = 10)
    findspikes = findspikes/1000
    nrSpikesIO = len(findspikes)
    setattr(SpikeTimes_s, IOnr, [findspikes[k] for k in range(0,nrSpikesIO)
                                 if findspikes[k]>=(start_t/1e3)])
    setattr(SpikeTimes_s, BA_IOnr, [findspikes[k] for k in range(0,nrSpikesIO) 
                                    if (findspikes[k]>=(start_t/1e3) and findspikes[k]<=(lengthSim/(2*1e3)))])
    setattr(SpikeTimes_s, AA_IOnr, [findspikes[k] for k in range(0,nrSpikesIO) 
                                    if findspikes[k]>=(t_beforeAdaptation/1e3)])
    